In [ ]:
#importar librerias
#peticion al servidor
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='http://espacialidades.cua.uam.mx/ojs/index.php/espacialidades/issue/archive?issuesPage=3#issues'
url_root= 'http://espacialidades.cua.uam.mx/ojs/index.php/espacialidades/issue/archive?issuesPage=3#issues'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
#Se realiza la busquea de la primeras etiquetas
#para obtener lo href de cada uno de lo volumenes o archivos de la revista.
box = soup.find('div', class_='issues media-list')
volumen=box.findAll('h2', class_='media-heading')
vol = [x.find('a').get('href')for x in volumen]
#vol=[urljoin(url_root,i) for i in vol]
vol

In [ ]:
#Se realiza un lista en la que la variable vol contiene los principales href
#con estos href obtenidos, se implemeta un ciclo de esto para con ello obtener
#los siguentes href de los arcticulos que se encuentran en cada uno de los volumenes o archivos
#para esto se realiza la misma busqueda que al principio.
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='sections')
    volumen1=soup1.findAll('h3',class_='media-heading')
    vol1 = [x.find('a').get('href')for x in volumen1]
    vol1=[urljoin(url_root,i) for i in vol1]
    vol2+=vol1

In [ ]:
vol2

In [ ]:
#En esta parte la funcion es de obtener links
def get_url_items(sopa, url): 
    box1 = soup1.find('div', class_='sections')
    volumen1=soup1.findAll('h3',class_='media-heading')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [ ]:
#Ahora se va acumulando cada uno de lo links e ir iterando en cada una
#de las páginas, para traer cada unos de los links que se van a ir scrapeando.
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
#Se hace un listado de los links de lo articulos que se obtuvieron.
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
#Se estaran tomado uno a uno los links para Scrapearlos.
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
#Mediante esta función resivi cada uno de los links, va ir aplicando caca uno de 
#los metodos, que nos van a obtener lo que estamos requiriendo.
def scraper_book(url):
    content_book={}# diccionario que no va acumulando la imformacion que vamos poteniddo para el tada frame
    r=requests.get(url)
    tr='Espacialidades. Revista de temas contemporáneos sobre lugares, política y cultura'
    a='V. Ciencias Sociales'
    te='Relación entre el espacio y la vida social'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('h1',class_='page-header').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
     #obtencion de Resumen
    try:
        des=s_item.find('div', class_='article-abstract').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    #obtencion de Abstract
    try:
        de=s_item.find('div', class_='panel-boody').get_text(strip=True)
        content_book['Abstract']=de#.replace("Abstract","")
    except AttributeError:
        content_book['Abstract']=None
    #obtener link Articulos con la url, donde se guarda lo href
    #donde se encuetran la informacion de los articulos.
    try:
        urlA=url
        content_book['Link Articulo']=urlA
    except AttributeError:
        content_book['Link Articulo']=None
    #obtener link PDF
    try:
        ancla_link=s_item.find('div', class_='download').find('a').get('href')
        content_book['Link PDF']=urljoin(url_root,ancla_link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [ ]:
#El ciclo hace un scrapeo de la cada uno de los articulos.
list_scraper=list_scraper[0:10]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
#En listado de datos_book que se obtuvo, se guarada en una variable
#y dicha variable que conteien la imformacion se convierte en un DataFrame.
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
#La imformacion conseguda se pasa a un CSV.
df_catalogo.to_csv('Revista200.2.csv', index=False)

Final Código